## **MACHINE LEARNING PRODUCTS (MLOps)**


***En este cuaderno jupiter se encuentra un producto minimo viable, de nuestros proyecto - "Sistema de Recomendacion del Mundial del 2026" solo vamos  a utilizar solo la data de Google Maps***

In [1]:
# Librerias
import pymysql
import mysql.connector
from mysql.connector import Error
from sqlalchemy import create_engine
from dotenv import load_dotenv
import pandas as pd
import os

In [2]:
# La data se encuentra en AWS vamos a conectar
# Cargamos el archivo .env
load_dotenv()

# Acceder a las variables entorno
password_bd = os.getenv("PASS_BD")
user_bd = os.getenv("USER_BD")
host_name = os.getenv("HOST_NAME")
name_bd = os.getenv("NAME_BD")

# Conectamos la base de datos


def BD_connection(host_name, user_bd, user_pass, name_bd):
    connection = None
    try:
        connection = mysql.connector.connect(
            host=host_name,
            user=user_bd,
            passwd=user_pass,
            database=name_bd,
            port=3306
        )
        print("MYSQL DATABASE connection succesful")
    except Error as err:
        print(f"Error: '{err}'")
    return connection

In [3]:
connection = BD_connection(host_name, user_bd, password_bd, name_bd)

MYSQL DATABASE connection succesful


In [4]:
# verifiquemos la conexion realizando una consulta simple
if connection:
    cursor = connection.cursor()
    cursor.execute("SELECT DATABASE()")
    db = cursor.fetchone()
    print(f"Conectado a la base de datos: {db[0]}")

Conectado a la base de datos: Google


In [5]:
# Funcion para obtener las tablas disponibles
def show_tables(connection):
    cursor = connection.cursor()
    cursor.execute("SHOW TABLES")
    tables = cursor.fetchall()
    for table in tables:
        print(table)

In [6]:
# Corremos la query
connection = BD_connection(host_name, user_bd, password_bd, name_bd)
show_tables(connection)

MYSQL DATABASE connection succesful
('sites',)
('users_google',)


In [7]:
# Funcion para describir la estructura de una tabla
def describe_table(connection, table_name):
    cursor = connection.cursor()
    query = f"DESCRIBE {table_name}"
    cursor.execute(query)
    columns = cursor.fetchall()
    for column in columns:
        print(column)

In [8]:
describe_table(connection, 'users_google')

('id_user', b'varchar(100)', 'NO', 'PRI', None, '')
('name', b'varchar(50)', 'NO', '', None, '')
('time_comment', b'date', 'YES', '', None, '')
('text_comment', b'varchar(1000)', 'YES', '', None, '')
('raiting', b'int', 'YES', '', None, '')
('pics', b'varchar(2048)', 'YES', '', None, '')
('time_resp', b'date', 'YES', '', None, '')
('text_resp', b'varchar(1000)', 'YES', '', None, '')
('gmap_id', b'varchar(100)', 'YES', 'MUL', None, '')
('state', b'varchar(30)', 'YES', '', None, '')


In [9]:
# Vamos a utilizar esta funcion para correr queries
def execute_query(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
        print("Query hecha con éxito")
    except Error as err:
        print("Error", err)

In [10]:
def execute_query(connection, query, fetch_data=False):
    """
    Ejecuta una consulta SQL en la base de datos conectada.

    Args:
    - connection: Conexión a la base de datos.
    - query: Consulta SQL a ejecutar.
    - fetch_data: Booleano para indicar si se deben recuperar los datos de la consulta.

    Returns:
    - Resultados de la consulta si fetch_data es True. None en caso contrario.
    """
    cursor = connection.cursor(buffered=True)  # Crear un cursor con buffering
    try:
        cursor.execute(query)  # Ejecutar la consulta

        if fetch_data:
            resultados = cursor.fetchall()  # Recuperar todos los resultados
            return resultados
        else:
            connection.commit()  # Confirmar la transacción
            print("Query hecha con éxito")

    except Error as err:
        print("Error:", err)  # Imprimir cualquier error ocurrido
    finally:
        cursor.close()  # Cerrar el cursor
        if connection.unread_result:
            connection.next_result()  # Limpiar cualquier resultado no leído
        connection.commit()  # Asegurar que cualquier resultado pendiente sea limpiado

In [11]:
# Verificar la conexión
if connection:
    print("Conexión exitosa a la base de datos")

    # Contar registros en `users_google`
    query = "SELECT COUNT(*) FROM users_google"
    resultado = execute_query(connection, query, fetch_data=True)

    if resultado:
        print(f"Número de registros en 'users_google': {resultado[0][0]}")

    # Obtener promedio de calificaciones
    query = "SELECT AVG(raiting) FROM users_google"
    resultado = execute_query(connection, query, fetch_data=True)

    if resultado:
        print(f"Promedio de calificaciones: {resultado[0][0]}")

Conexión exitosa a la base de datos
Número de registros en 'users_google': 0
Promedio de calificaciones: None
